# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-822bf09967-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# History Workflow Series ("Ancient Civilization explorative search") 

Consider the following exploratory information need:

> Investigate the Ancient Egypt and the related archeological explorations and findings.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q13444093`| Meidum Pyramid    | node |
| `wd:Q41127`| Poseidon   | node |





Also consider

```
wd:Q13444093 ?p ?obj .
```

is the BGP to retrieve all **properties of Meidum Pyramid**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.



The workload should:


1. Identify the BGP for Ancient Egypt

2. Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

3. Find all the ancient civilizations. Return the IRI and name of the civilization and also their duration (in number of years) if it is available.

4. Identify the BGP for archeologist.

5. For each ancient civilization find the number of archeologists who worked on it (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

6. Find the BGP for deity.

7. List all the twelve olympian greek deities (the result set must a list of 12 elements of couples of deity IRI and label).

8. Compare ancient Greece with ancient Egypt and find how many deities each one has.


## Task 1
Identify the BGP for Ancient Egypt

In [2]:
# getting object/data properties of node 'medium pyramid'
queryString = """
SELECT DISTINCT ?property ?label
WHERE
{
   wd:Q13444093 ?property ?object.
   
   ?property sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P131'), ('label', 'located in the administrative territorial entity')]
[('property', 'http://www.wikidata.org/prop/direct/P138'), ('label', 'named after')]
[('property', 'http://www.wikidata.org/prop/direct/P1481'), ('label', 'vici.org ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1566'), ('label', 'GeoNames ID')]
[('property', 'http://www.wikidata.org/prop/direct/P17'), ('label', 'country')]
[('property', 'http://www.wikidata.org/prop/direct/P18'), ('label', 'image')]
[('property', 'http://www.wikidata.org/prop/direct/P186'), ('label', 'made from material')]
[('property', 'http://www.wikidata.org/prop/direct/P2044'), ('label', 'elevation above sea level')]
[('property', 'http://www.wikidata.org/prop/direct/P2048'), ('label', 'height')]
[('property', 'http://www.wikidata.org/prop/direct/P214'), ('label', 'VIAF ID')]
[('property', 'http://www.wikidata.org/prop/direct/P227'), ('label', 'GND ID')]
[('property', 'http:/

In [3]:
# checking 'located in the administrative territorial entity' property of node 'medium pyramid'
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q13444093 wdt:P131 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q30683'), ('label', 'Beni Suef Governorate')]
1


In [4]:
# checking 'instance of' property of node 'medium pyramid'
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q13444093 wdt:P31 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q1456099'), ('label', 'step pyramid')]
1


In [5]:
# getting object/data properties of node 'step pyramid'
queryString = """
SELECT DISTINCT ?property ?label
WHERE
{
   wd:Q1456099 ?property ?object.
   
   ?property sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P18'), ('label', 'image')]
[('property', 'http://www.wikidata.org/prop/direct/P279'), ('label', 'subclass of')]
[('property', 'http://www.wikidata.org/prop/direct/P373'), ('label', 'Commons category')]
[('property', 'http://www.wikidata.org/prop/direct/P4212'), ('label', 'PACTOLS thesaurus ID')]
[('property', 'http://www.wikidata.org/prop/direct/P461'), ('label', 'opposite of')]
[('property', 'http://www.wikidata.org/prop/direct/P646'), ('label', 'Freebase ID')]
6


In [6]:
# checking 'comissioned by' property of node 'medium pyramid'
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q13444093 wdt:P88 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q189371'), ('label', 'Sneferu')]
[('object', 'http://www.wikidata.org/entity/Q220240'), ('label', 'Huni')]
2


In [7]:
# getting object/data properties of node 'Sneferu'
queryString = """
SELECT DISTINCT ?property ?label
WHERE
{
   wd:Q189371 ?property ?object.
   
   ?property sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P106'), ('label', 'occupation')]
[('property', 'http://www.wikidata.org/prop/direct/P119'), ('label', 'place of burial')]
[('property', 'http://www.wikidata.org/prop/direct/P1263'), ('label', 'NNDB people ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1296'), ('label', 'Gran Enciclopèdia Catalana ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1309'), ('label', 'EGAXA ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1343'), ('label', 'described by source')]
[('property', 'http://www.wikidata.org/prop/direct/P1417'), ('label', 'Encyclopædia Britannica Online ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1711'), ('label', 'British Museum person or institution ID')]
[('property', 'http://www.wikidata.org/prop/direct/P18'), ('label', 'image')]
[('property', 'http://www.wikidata.org/prop/direct/P19'), ('label', 'place of birth')]
[('property', 'http://www.wikidata.org/prop/direct/P20'), ('la

In [8]:
# checking 'time period' property of node 'Sneferu'
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q189371 wdt:P2348 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q177819'), ('label', 'Old Kingdom of Egypt')]
[('object', 'http://www.wikidata.org/entity/Q11768'), ('label', 'Ancient Egypt')]
2


Final query for this task

In [9]:
# write your final query
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q13444093 wdt:P88 ?comissioner.
   ?comissioner wdt:P2348 ?object.
   
   ?object sc:name ?label.
}
ORDER BY ASC(?label)
LIMIT 1
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q11768'), ('label', 'Ancient Egypt')]
1


## Task 2
Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

In [10]:
# write your queries
# getting object/data properties of node 'ancient egypt'
queryString = """
SELECT DISTINCT ?property ?label
WHERE
{
   wd:Q11768 ?property ?object.
   
   ?property sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1036'), ('label', 'Dewey Decimal Classification')]
[('property', 'http://www.wikidata.org/prop/direct/P1151'), ('label', "topic's main Wikimedia portal")]
[('property', 'http://www.wikidata.org/prop/direct/P1282'), ('label', 'OpenStreetMap tag or key')]
[('property', 'http://www.wikidata.org/prop/direct/P1343'), ('label', 'described by source')]
[('property', 'http://www.wikidata.org/prop/direct/P1417'), ('label', 'Encyclopædia Britannica Online ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1465'), ('label', 'category for people who died here')]
[('property', 'http://www.wikidata.org/prop/direct/P155'), ('label', 'follows')]
[('property', 'http://www.wikidata.org/prop/direct/P1584'), ('label', 'Pleiades ID')]
[('property', 'http://www.wikidata.org/prop/direct/P18'), ('label', 'image')]
[('property', 'http://www.wikidata.org/prop/direct/P214'), ('label', 'VIAF ID')]
[('property', 'http://www.wikidata.org/prop/direct

In [11]:
# checking 'occupation' property of node 'Sneferu' that we found before
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q189371 wdt:P106 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q372436'), ('label', 'statesperson')]
1


In [12]:
# checking 'instance of' property of node 'Sneferu'
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q189371 wdt:P31 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q5'), ('label', 'human')]
1


In [13]:
# checking 'noble title' property of node 'Sneferu'
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q189371 wdt:P97 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q37110'), ('label', 'pharaoh')]
1


In [14]:
# getting all pharaohs
queryString = """
SELECT DISTINCT ?subject ?label
WHERE
{
   ?subject wdt:P97 wd:Q37110.
   
   ?subject sc:name ?label.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('subject', 'http://www.wikidata.org/entity/Q151828'), ('label', 'Djet')]
[('subject', 'http://www.wikidata.org/entity/Q152375'), ('label', 'Djer')]
[('subject', 'http://www.wikidata.org/entity/Q19248'), ('label', 'Senusret III')]
[('subject', 'http://www.wikidata.org/entity/Q230774'), ('label', 'Nitocris')]
[('subject', 'http://www.wikidata.org/entity/Q263218'), ('label', 'Djedkhonsuefankh')]
[('subject', 'http://www.wikidata.org/entity/Q269265'), ('label', 'Pepi I Meryre')]
[('subject', 'http://www.wikidata.org/entity/Q320908'), ('label', 'Merenre Nemtyemsaf II')]
[('subject', 'http://www.wikidata.org/entity/Q336395'), ('label', 'Smendes')]
[('subject', 'http://www.wikidata.org/entity/Q380676'), ('label', 'Psusennes I')]
[('subject', 'http://www.wikidata.org/entity/Q428633'), ('label', 'Piankh')]
[('subject', 'http://www.wikidata.org/entity/Q434877'), ('label', 'Amenemnisu')]
[('subject', 'http://www.wikidata.org/entity/Q440496'), ('label', 'Khyan')]
[('subject', 'http://www

In [15]:
# checking 'date of birth' property of all pharaohs
queryString = """
SELECT DISTINCT ?subject ?label ?date
WHERE
{
   ?subject wdt:P97 wd:Q37110.
   ?subject wdt:P569 ?date.
   
   ?subject sc:name ?label.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('subject', 'http://www.wikidata.org/entity/Q151828'), ('label', 'Djet'), ('date', '-3000-01-01T00:00:00Z')]
[('subject', 'http://www.wikidata.org/entity/Q152375'), ('label', 'Djer'), ('date', '-3001-01-01T00:00:00Z')]
[('subject', 'http://www.wikidata.org/entity/Q19248'), ('label', 'Senusret III'), ('date', '-1950-01-01T00:00:00Z')]
[('subject', 'http://www.wikidata.org/entity/Q230774'), ('label', 'Nitocris'), ('date', '-2250-01-01T00:00:00Z')]
[('subject', 'http://www.wikidata.org/entity/Q263218'), ('label', 'Djedkhonsuefankh'), ('date', '-1100-01-01T00:00:00Z')]
[('subject', 'http://www.wikidata.org/entity/Q269265'), ('label', 'Pepi I Meryre'), ('date', '-2400-01-01T00:00:00Z')]
[('subject', 'http://www.wikidata.org/entity/Q320908'), ('label', 'Merenre Nemtyemsaf II'), ('date', '-2300-01-01T00:00:00Z')]
[('subject', 'http://www.wikidata.org/entity/Q336395'), ('label', 'Smendes'), ('date', '-1200-01-01T00:00:00Z')]
[('subject', 'http://www.wikidata.org/entity/Q380676'), ('la

In [16]:
# getting all pharaohs born between 1000 BCE and 1500 BCE
# 901-1000: 10. century BCE
# 1001-1100: 11. century BCE
# ...
# 1401-1500: 15. century BCE
queryString = """
SELECT ?subject ?label ?date
WHERE
{
   ?subject wdt:P97 wd:Q37110.
   ?subject wdt:P569 ?date.
   
   FILTER (datatype(?date) = xsd:dateTime)
   FILTER ("-1500-01-01T00:00:00Z"^^xsd:dateTime <= ?date && ?date <= "-901-01-01T00:00:00Z"^^xsd:dateTime)
   
   ?subject sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('subject', 'http://www.wikidata.org/entity/Q263218'), ('label', 'Djedkhonsuefankh'), ('date', '-1100-01-01T00:00:00Z')]
[('subject', 'http://www.wikidata.org/entity/Q336395'), ('label', 'Smendes'), ('date', '-1200-01-01T00:00:00Z')]
[('subject', 'http://www.wikidata.org/entity/Q380676'), ('label', 'Psusennes I'), ('date', '-1100-01-01T00:00:00Z')]
[('subject', 'http://www.wikidata.org/entity/Q428633'), ('label', 'Piankh'), ('date', '-1200-01-01T00:00:00Z')]
[('subject', 'http://www.wikidata.org/entity/Q434877'), ('label', 'Amenemnisu'), ('date', '-1100-01-01T00:00:00Z')]
[('subject', 'http://www.wikidata.org/entity/Q705367'), ('label', 'Menkheperre'), ('date', '-1100-01-01T00:00:00Z')]
[('subject', 'http://www.wikidata.org/entity/Q459153'), ('label', 'Osorkon II'), ('date', '-1000-01-01T00:00:00Z')]
[('subject', 'http://www.wikidata.org/entity/Q146029'), ('label', 'Ay'), ('date', '-1400-01-01T00:00:00Z')]
[('subject', 'http://www.wikidata.org/entity/Q517662'), ('label', 'Yuya

In [17]:
# checking 'child' property of all pharaohs
queryString = """
SELECT DISTINCT ?pharaoh (COUNT(?object) AS ?count)
WHERE
{
   ?pharaoh wdt:P97 wd:Q37110.
   ?pharaoh wdt:P40 ?object.
}
GROUP BY ?pharaoh
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('pharaoh', 'http://www.wikidata.org/entity/Q373521'), ('count', '1')]
[('pharaoh', 'http://www.wikidata.org/entity/Q199861'), ('count', '9')]
[('pharaoh', 'http://www.wikidata.org/entity/Q291895'), ('count', '1')]
[('pharaoh', 'http://www.wikidata.org/entity/Q316274'), ('count', '2')]
[('pharaoh', 'http://www.wikidata.org/entity/Q1532'), ('count', '1')]
[('pharaoh', 'http://www.wikidata.org/entity/Q18771'), ('count', '2')]
[('pharaoh', 'http://www.wikidata.org/entity/Q158043'), ('count', '4')]
[('pharaoh', 'http://www.wikidata.org/entity/Q220252'), ('count', '3')]
[('pharaoh', 'http://www.wikidata.org/entity/Q175775'), ('count', '1')]
[('pharaoh', 'http://www.wikidata.org/entity/Q21694324'), ('count', '1')]
[('pharaoh', 'http://www.wikidata.org/entity/Q3345'), ('count', '3')]
[('pharaoh', 'http://www.wikidata.org/entity/Q878396'), ('count', '1')]
[('pharaoh', 'http://www.wikidata.org/entity/Q2513638'), ('count', '2')]
[('pharaoh', 'http://www.wikidata.org/entity/Q244985'), ('

Final query for this task

In [18]:
# write your final query
queryString = """
SELECT ?pharaoh_iri ?label (COUNT(?child) AS ?num_child)
WHERE
{
   ?pharaoh_iri wdt:P97 wd:Q37110.
   ?pharaoh_iri wdt:P569 ?date.
   ?pharaoh_iri wdt:P40 ?child.
   
   FILTER (datatype(?date) = xsd:dateTime)
   FILTER ("-1500-01-01T00:00:00Z"^^xsd:dateTime <= ?date && ?date <= "-901-01-01T00:00:00Z"^^xsd:dateTime)
   
   ?pharaoh_iri sc:name ?label.
}
GROUP BY ?pharaoh_iri ?label
"""

print("Results")
x=run_query(queryString)

Results
[('pharaoh_iri', 'http://www.wikidata.org/entity/Q1532'), ('label', 'Ramesses IV'), ('num_child', '1')]
[('pharaoh_iri', 'http://www.wikidata.org/entity/Q158043'), ('label', 'Merneptah'), ('num_child', '4')]
[('pharaoh_iri', 'http://www.wikidata.org/entity/Q444965'), ('label', 'Pinedjem I'), ('num_child', '8')]
[('pharaoh_iri', 'http://www.wikidata.org/entity/Q165989'), ('label', 'Psusennes II'), ('num_child', '1')]
[('pharaoh_iri', 'http://www.wikidata.org/entity/Q312402'), ('label', 'Setnakhte'), ('num_child', '2')]
[('pharaoh_iri', 'http://www.wikidata.org/entity/Q428633'), ('label', 'Piankh'), ('num_child', '1')]
[('pharaoh_iri', 'http://www.wikidata.org/entity/Q258452'), ('label', 'Pinedjem II'), ('num_child', '2')]
[('pharaoh_iri', 'http://www.wikidata.org/entity/Q157985'), ('label', 'Thutmose IV'), ('num_child', '6')]
[('pharaoh_iri', 'http://www.wikidata.org/entity/Q146029'), ('label', 'Ay'), ('num_child', '2')]
[('pharaoh_iri', 'http://www.wikidata.org/entity/Q517662')

## Task 3
Find all the ancient civilizations. Return the IRI and name of the civilization and also their duration (in number of years) if it is available.

In [19]:
# write your queries
# checking 'instance of' property of node 'ancient egypt'
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q11768 wdt:P31 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q28171280'), ('label', 'ancient civilization')]
[('object', 'http://www.wikidata.org/entity/Q3024240'), ('label', 'historical country')]
[('object', 'http://www.wikidata.org/entity/Q3502482'), ('label', 'cultural region')]
[('object', 'http://www.wikidata.org/entity/Q1620908'), ('label', 'historical region')]
4


In [20]:
# getting all instances of ancient civilization
queryString = """
SELECT DISTINCT ?subject ?label
WHERE
{
   ?subject wdt:P31 wd:Q28171280.
   
   ?subject sc:name ?label.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('subject', 'http://www.wikidata.org/entity/Q35355'), ('label', 'Sumer')]
[('subject', 'http://www.wikidata.org/entity/Q25558247'), ('label', 'Chinese civilization')]
[('subject', 'http://www.wikidata.org/entity/Q38060'), ('label', 'Gaul')]
[('subject', 'http://www.wikidata.org/entity/Q134178'), ('label', 'Minoan civilization')]
[('subject', 'http://www.wikidata.org/entity/Q173082'), ('label', 'Dacia')]
[('subject', 'http://www.wikidata.org/entity/Q2429397'), ('label', 'Ancient Carthage')]
[('subject', 'http://www.wikidata.org/entity/Q3327860'), ('label', 'Celtic kingdom of Ireland')]
[('subject', 'http://www.wikidata.org/entity/Q193748'), ('label', 'history of ancient Israel and Judah')]
[('subject', 'http://www.wikidata.org/entity/Q11768'), ('label', 'Ancient Egypt')]
[('subject', 'http://www.wikidata.org/entity/Q11772'), ('label', 'Ancient Greece')]
[('subject', 'http://www.wikidata.org/entity/Q41741'), ('label', 'Thrace')]
[('subject', 'http://www.wikidata.org/entity/Q1289

In [21]:
# checking duration using 'start time' and 'end time' properties of node 'ancient egypt'
queryString = """
SELECT ?start ?end ?duration
WHERE
{
   wd:Q11768 wdt:P580 ?start.
   wd:Q11768 wdt:P582 ?end.
   
   BIND (year(?end) - year(?start) AS ?duration)
}
"""

print("Results")
x=run_query(queryString)

Results
[('start', '-4000-01-01T00:00:00Z'), ('end', '0395-01-01T00:00:00Z'), ('duration', '4395')]
1


In [22]:
# checking duration using 'start time' and 'end time' properties of node 'minoan civilization' that found earlier
queryString = """
SELECT ?start ?end ?duration
WHERE
{
   wd:Q134178 wdt:P580 ?start.
   wd:Q134178 wdt:P582 ?end.
   
   BIND (year(?end) - year(?start) AS ?duration)
}
"""

print("Results")
x=run_query(queryString)

Results
[('start', '-3299-01-01T00:00:00Z'), ('end', '-1099-01-01T00:00:00Z'), ('duration', '2200')]
[('start', '-2000-01-01T00:00:00Z'), ('end', '-1099-01-01T00:00:00Z'), ('duration', '901')]
2


Final query for this task

In [23]:
# write your final query

# in the output minoan civilization has two durations because of two different start times stated
queryString = """
SELECT ?civilization_iri ?label ?duration
WHERE
{
   ?civilization_iri wdt:P31 wd:Q28171280.
   
   OPTIONAL 
   {
       ?civilization_iri wdt:P580 ?start.
       ?civilization_iri wdt:P582 ?end.

       BIND (year(?end) - year(?start) AS ?duration)
   }
   
   ?civilization_iri sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('civilization_iri', 'http://www.wikidata.org/entity/Q134178'), ('label', 'Minoan civilization'), ('duration', '901')]
[('civilization_iri', 'http://www.wikidata.org/entity/Q134178'), ('label', 'Minoan civilization'), ('duration', '2200')]
[('civilization_iri', 'http://www.wikidata.org/entity/Q11768'), ('label', 'Ancient Egypt'), ('duration', '4395')]
[('civilization_iri', 'http://www.wikidata.org/entity/Q11772'), ('label', 'Ancient Greece'), ('duration', '1800')]
[('civilization_iri', 'http://www.wikidata.org/entity/Q42534'), ('label', 'Indus Valley Civilization'), ('duration', '1200')]
[('civilization_iri', 'http://www.wikidata.org/entity/Q1551054'), ('label', 'Israelite period'), ('duration', '614')]
[('civilization_iri', 'http://www.wikidata.org/entity/Q35355'), ('label', 'Sumer')]
[('civilization_iri', 'http://www.wikidata.org/entity/Q25558247'), ('label', 'Chinese civilization')]
[('civilization_iri', 'http://www.wikidata.org/entity/Q38060'), ('label', 'Gaul')]
[('civili

## Task 4
Identify the BGP for archeologist.

In [24]:
# write your queries
# checking 'studied by' property of node 'ancient egypt'
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q11768 wdt:P2579 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q4752891'), ('label', 'ancient Near Eastern studies')]
1


In [25]:
# getting object/data properties of node 'ancient near eastern studies'
queryString = """
SELECT DISTINCT ?property ?label
WHERE
{
   wd:Q4752891 ?property ?object.
   
   ?property sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P2578'), ('label', 'studies')]
[('property', 'http://www.wikidata.org/prop/direct/P279'), ('label', 'subclass of')]
[('property', 'http://www.wikidata.org/prop/direct/P31'), ('label', 'instance of')]
[('property', 'http://www.wikidata.org/prop/direct/P527'), ('label', 'has part')]
[('property', 'http://www.wikidata.org/prop/direct/P646'), ('label', 'Freebase ID')]
5


In [26]:
# checking 'subclass of' property of node 'ancient near eastern studies'
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q4752891 wdt:P279 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q23019185'), ('label', 'Near Eastern studies')]
1


In [27]:
# checking 'has part' property of node 'ancient near eastern studies'
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q4752891 wdt:P527 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q4785363'), ('label', 'genetic history of the Middle East')]
[('object', 'http://www.wikidata.org/entity/Q145903'), ('label', 'egyptology')]
[('object', 'http://www.wikidata.org/entity/Q156859'), ('label', 'Assyriology')]
[('object', 'http://www.wikidata.org/entity/Q258212'), ('label', 'Near Eastern archaeology')]
[('object', 'http://www.wikidata.org/entity/Q491481'), ('label', 'Hittitology')]
5


In [28]:
# getting object/data properties of node 'egyptology'
queryString = """
SELECT DISTINCT ?property ?label
WHERE
{
   wd:Q145903 ?property ?object.
   
   ?property sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1014'), ('label', 'Art & Architecture Thesaurus ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1051'), ('label', 'PSH ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1245'), ('label', 'OmegaWiki Defined Meaning')]
[('property', 'http://www.wikidata.org/prop/direct/P1296'), ('label', 'Gran Enciclopèdia Catalana ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1343'), ('label', 'described by source')]
[('property', 'http://www.wikidata.org/prop/direct/P1417'), ('label', 'Encyclopædia Britannica Online ID')]
[('property', 'http://www.wikidata.org/prop/direct/P18'), ('label', 'image')]
[('property', 'http://www.wikidata.org/prop/direct/P227'), ('label', 'GND ID')]
[('property', 'http://www.wikidata.org/prop/direct/P244'), ('label', 'Library of Congress authority ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2578'), ('label', 'studies')]
[('property', 'http://www.wikidata.org/prop/direct/P2

In [29]:
# checking 'studies' property of node 'egyptology'
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q145903 wdt:P2578 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q11768'), ('label', 'Ancient Egypt')]
[('object', 'http://www.wikidata.org/entity/Q135028'), ('label', 'Nubia')]
2


In [30]:
# checking 'subclass of' property of node 'egyptology'
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q145903 wdt:P279 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q23498'), ('label', 'archaeology')]
[('object', 'http://www.wikidata.org/entity/Q4752891'), ('label', 'ancient Near Eastern studies')]
2


In [31]:
# getting object/data properties of node 'archeology'
queryString = """
SELECT DISTINCT ?property ?label
WHERE
{
   wd:Q23498 ?property ?object.
   
   ?property sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1014'), ('label', 'Art & Architecture Thesaurus ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1036'), ('label', 'Dewey Decimal Classification')]
[('property', 'http://www.wikidata.org/prop/direct/P1051'), ('label', 'PSH ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1151'), ('label', "topic's main Wikimedia portal")]
[('property', 'http://www.wikidata.org/prop/direct/P1225'), ('label', 'U.S. National Archives Identifier')]
[('property', 'http://www.wikidata.org/prop/direct/P1245'), ('label', 'OmegaWiki Defined Meaning')]
[('property', 'http://www.wikidata.org/prop/direct/P1343'), ('label', 'described by source')]
[('property', 'http://www.wikidata.org/prop/direct/P1417'), ('label', 'Encyclopædia Britannica Online ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1424'), ('label', "topic's main template")]
[('property', 'http://www.wikidata.org/prop/direct/P1438'), ('label', 'Jewish Encyclopedia I

In [32]:
# checking 'practiced by' property of node 'archeology'
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q23498 wdt:P3095 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q3621491'), ('label', 'archaeologist')]
1


Final query for this task

In [33]:
# write your final query
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q23498 wdt:P3095 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q3621491'), ('label', 'archaeologist')]
1


## Task 5
For each ancient civilization find the number of archeologists who worked on it (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

In [34]:
# write your queries
# getting all instances of ancient civilization
queryString = """
SELECT DISTINCT ?subject ?label
WHERE
{
   ?subject wdt:P31 wd:Q28171280.
   
   ?subject sc:name ?label.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('subject', 'http://www.wikidata.org/entity/Q35355'), ('label', 'Sumer')]
[('subject', 'http://www.wikidata.org/entity/Q25558247'), ('label', 'Chinese civilization')]
[('subject', 'http://www.wikidata.org/entity/Q38060'), ('label', 'Gaul')]
[('subject', 'http://www.wikidata.org/entity/Q134178'), ('label', 'Minoan civilization')]
[('subject', 'http://www.wikidata.org/entity/Q173082'), ('label', 'Dacia')]
[('subject', 'http://www.wikidata.org/entity/Q2429397'), ('label', 'Ancient Carthage')]
[('subject', 'http://www.wikidata.org/entity/Q3327860'), ('label', 'Celtic kingdom of Ireland')]
[('subject', 'http://www.wikidata.org/entity/Q193748'), ('label', 'history of ancient Israel and Judah')]
[('subject', 'http://www.wikidata.org/entity/Q11768'), ('label', 'Ancient Egypt')]
[('subject', 'http://www.wikidata.org/entity/Q11772'), ('label', 'Ancient Greece')]
[('subject', 'http://www.wikidata.org/entity/Q41741'), ('label', 'Thrace')]
[('subject', 'http://www.wikidata.org/entity/Q1289

In [35]:
# checking 'studied by' property of each ancient civilization
queryString = """
SELECT DISTINCT ?subject ?label_subject ?object ?label_object
WHERE
{
   ?subject wdt:P31 wd:Q28171280.
   ?subject wdt:P2579 ?object.
   
   ?subject sc:name ?label_subject.
   ?object sc:name ?label_object.
}
"""

print("Results")
x=run_query(queryString)

Results
[('subject', 'http://www.wikidata.org/entity/Q11772'), ('label_subject', 'Ancient Greece'), ('object', 'http://www.wikidata.org/entity/Q841090'), ('label_object', 'classics')]
[('subject', 'http://www.wikidata.org/entity/Q32047'), ('label_subject', 'Illyria'), ('object', 'http://www.wikidata.org/entity/Q12957521'), ('label_object', 'Illyrology')]
[('subject', 'http://www.wikidata.org/entity/Q41741'), ('label_subject', 'Thrace'), ('object', 'http://www.wikidata.org/entity/Q3110778'), ('label_object', 'thracology')]
[('subject', 'http://www.wikidata.org/entity/Q11768'), ('label_subject', 'Ancient Egypt'), ('object', 'http://www.wikidata.org/entity/Q4752891'), ('label_object', 'ancient Near Eastern studies')]
[('subject', 'http://www.wikidata.org/entity/Q11772'), ('label_subject', 'Ancient Greece'), ('object', 'http://www.wikidata.org/entity/Q435608'), ('label_object', 'ancient history')]
[('subject', 'http://www.wikidata.org/entity/Q11772'), ('label_subject', 'Ancient Greece'), (

In [36]:
# getting object/data properties of node 'human'
queryString = """
SELECT DISTINCT ?property ?label
WHERE
{
   wd:Q5 ?property ?object.
   
   ?property sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P5806'), ('label', 'SNOMED CT identifier')]
[('property', 'http://www.wikidata.org/prop/direct/P1036'), ('label', 'Dewey Decimal Classification')]
[('property', 'http://www.wikidata.org/prop/direct/P1051'), ('label', 'PSH ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1056'), ('label', 'product or material produced')]
[('property', 'http://www.wikidata.org/prop/direct/P1225'), ('label', 'U.S. National Archives Identifier')]
[('property', 'http://www.wikidata.org/prop/direct/P1245'), ('label', 'OmegaWiki Defined Meaning')]
[('property', 'http://www.wikidata.org/prop/direct/P1256'), ('label', 'Iconclass notation')]
[('property', 'http://www.wikidata.org/prop/direct/P129'), ('label', 'physically interacts with')]
[('property', 'http://www.wikidata.org/prop/direct/P1296'), ('label', 'Gran Enciclopèdia Catalana ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1343'), ('label', 'described by source')]
[('property'

In [37]:
# checking 'properties for this type' property of node 'human'
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q5 wdt:P1963 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/P1317'), ('label', 'floruit')]
[('object', 'http://www.wikidata.org/entity/P1477'), ('label', 'birth name')]
[('object', 'http://www.wikidata.org/entity/P1559'), ('label', 'name in native language')]
[('object', 'http://www.wikidata.org/entity/P509'), ('label', 'cause of death')]
[('object', 'http://www.wikidata.org/entity/P3373'), ('label', 'sibling')]
[('object', 'http://www.wikidata.org/entity/P119'), ('label', 'place of burial')]
[('object', 'http://www.wikidata.org/entity/P937'), ('label', 'work location')]
[('object', 'http://www.wikidata.org/entity/P101'), ('label', 'field of work')]
[('object', 'http://www.wikidata.org/entity/P140'), ('label', 'religion')]
[('object', 'http://www.wikidata.org/entity/P18'), ('label', 'image')]
[('object', 'http://www.wikidata.org/entity/P25'), ('label', 'mother')]
[('object', 'http://www.wikidata.org/entity/P26'), ('label', 'spouse')]
[('object', 'http://www.wikidata.org/entity/P2048'), ('label

In [38]:
# getting all instances of human that their 'occupation' property is archaeologist
# no result
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q5 wdt:P106 ?object.
   
   FILTER REGEX(?label, "archaeologist")
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [39]:
# checking 'field of work' property of node 'human'
# no result
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q5 wdt:P101 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [40]:
# getting all instances of human that their feild of work is any of ancient civilizations
queryString = """
SELECT ?subject ?label
WHERE
{
   ?civilization wdt:P31 wd:Q28171280.
   ?subject wdt:P101 ?civilization.
   
   ?subject sc:name ?label.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('subject', 'http://www.wikidata.org/entity/Q499357'), ('label', 'Andreas Luther')]
[('subject', 'http://www.wikidata.org/entity/Q2732854'), ('label', 'Epigraphical Museum')]
[('subject', 'http://www.wikidata.org/entity/Q2342814'), ('label', 'Alexandre Bertrand')]
[('subject', 'http://www.wikidata.org/entity/Q4446512'), ('label', 'Igor Surikov')]
[('subject', 'http://www.wikidata.org/entity/Q1985476'), ('label', 'Charles Thomas Newton')]
[('subject', 'http://www.wikidata.org/entity/Q17122314'), ('label', 'Greek Study Association')]
[('subject', 'http://www.wikidata.org/entity/Q21032203'), ('label', 'Aenne Ohnesorg')]
[('subject', 'http://www.wikidata.org/entity/Q2976497'), ('label', 'Classical Association')]
[('subject', 'http://www.wikidata.org/entity/Q1523942'), ('label', 'Gilbert Charles-Picard')]
[('subject', 'http://www.wikidata.org/entity/Q64563606'), ('label', "Vincent Robin d'Arba Desborough")]
[('subject', 'http://www.wikidata.org/entity/Q78158922'), ('label', 'Patric

Final query for this task

In [41]:
# write your final query
queryString = """
SELECT ?civilization_iri ?label (COUNT(?archeologist) AS ?num_archeologist)
WHERE
{
   ?civilization_iri wdt:P31 wd:Q28171280.
   
   OPTIONAL 
   {
       ?archeologist wdt:P101 ?civilization_iri.
   }
   
   ?civilization_iri sc:name ?label.
}
GROUP BY ?civilization_iri ?label
"""

print("Results")
x=run_query(queryString)

Results
[('civilization_iri', 'http://www.wikidata.org/entity/Q193748'), ('label', 'history of ancient Israel and Judah'), ('num_archeologist', '2')]
[('civilization_iri', 'http://www.wikidata.org/entity/Q11767'), ('label', 'Mesopotamia'), ('num_archeologist', '3')]
[('civilization_iri', 'http://www.wikidata.org/entity/Q11772'), ('label', 'Ancient Greece'), ('num_archeologist', '29')]
[('civilization_iri', 'http://www.wikidata.org/entity/Q173082'), ('label', 'Dacia'), ('num_archeologist', '0')]
[('civilization_iri', 'http://www.wikidata.org/entity/Q1364601'), ('label', 'Gallo-Roman culture'), ('num_archeologist', '0')]
[('civilization_iri', 'http://www.wikidata.org/entity/Q768212'), ('label', 'Turco-Mongol'), ('num_archeologist', '0')]
[('civilization_iri', 'http://www.wikidata.org/entity/Q1551054'), ('label', 'Israelite period'), ('num_archeologist', '2')]
[('civilization_iri', 'http://www.wikidata.org/entity/Q28405'), ('label', 'Gojoseon'), ('num_archeologist', '0')]
[('civilization_

## Task 6
Find the BGP for deity.

In [42]:
# write your queries
# getting object/data properties of node 'Poseidon'
queryString = """
SELECT DISTINCT ?property ?label
WHERE
{
   wd:Q41127 ?property ?object.
   
   ?property sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1296'), ('label', 'Gran Enciclopèdia Catalana ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1299'), ('label', 'depicted by')]
[('property', 'http://www.wikidata.org/prop/direct/P1343'), ('label', 'described by source')]
[('property', 'http://www.wikidata.org/prop/direct/P1417'), ('label', 'Encyclopædia Britannica Online ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1441'), ('label', 'present in work')]
[('property', 'http://www.wikidata.org/prop/direct/P18'), ('label', 'image')]
[('property', 'http://www.wikidata.org/prop/direct/P1871'), ('label', 'CERL Thesaurus ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1889'), ('label', 'different from')]
[('property', 'http://www.wikidata.org/prop/direct/P21'), ('label', 'sex or gender')]
[('property', 'http://www.wikidata.org/prop/direct/P214'), ('label', 'VIAF ID')]
[('property', 'http://www.wikidata.org/prop/direct/P22'), ('label', 'father')]
[('pr

In [43]:
# checking 'instance of' property of node 'Poseidon'
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q41127 wdt:P31 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q1916821'), ('label', 'water deity')]
[('object', 'http://www.wikidata.org/entity/Q22989102'), ('label', 'Greek deity')]
2


In [44]:
# checking the 'instance of' property of node 'greek deity'
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q22989102 wdt:P31 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [45]:
# checking the 'subclass of' property of node 'greek deity'
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q22989102 wdt:P279 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q95074'), ('label', 'fictional character')]
[('object', 'http://www.wikidata.org/entity/Q22988604'), ('label', 'mythological Greek character')]
[('object', 'http://www.wikidata.org/entity/Q178885'), ('label', 'deity')]
3


Final query for this task

In [46]:
# write your final query
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q41127 wdt:P31 ?greek_deity.
   ?greek_deity wdt:P279 ?object.
   
   ?object sc:name ?label.
}
ORDER BY ASC(?label)
LIMIT 1
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q178885'), ('label', 'deity')]
1


## Task 7
List all the twelve olympian greek deities (the result set must a list of 12 elements of couples of deity IRI and label).

In [47]:
# write your queries
# checkin 'part of' property of node 'Poseidon'
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q41127 wdt:P361 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q101609'), ('label', 'Twelve Olympians')]
1


In [48]:
# getting object/data properties of node 'twelve olympians'
queryString = """
SELECT DISTINCT ?property ?label
WHERE
{
   wd:Q101609 ?property ?object.
   
   ?property sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1343'), ('label', 'described by source')]
[('property', 'http://www.wikidata.org/prop/direct/P18'), ('label', 'image')]
[('property', 'http://www.wikidata.org/prop/direct/P227'), ('label', 'GND ID')]
[('property', 'http://www.wikidata.org/prop/direct/P279'), ('label', 'subclass of')]
[('property', 'http://www.wikidata.org/prop/direct/P2924'), ('label', 'Great Russian Encyclopedia Online ID')]
[('property', 'http://www.wikidata.org/prop/direct/P31'), ('label', 'instance of')]
[('property', 'http://www.wikidata.org/prop/direct/P3417'), ('label', 'Quora topic ID')]
[('property', 'http://www.wikidata.org/prop/direct/P373'), ('label', 'Commons category')]
[('property', 'http://www.wikidata.org/prop/direct/P4212'), ('label', 'PACTOLS thesaurus ID')]
[('property', 'http://www.wikidata.org/prop/direct/P443'), ('label', 'pronunciation audio')]
[('property', 'http://www.wikidata.org/prop/direct/P5063'), ('label', 'Interlingual Index ID'

In [49]:
# checking 'has part' property of node 'tweleve olympians'
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q101609 wdt:P527 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q37340'), ('label', 'Apollo')]
[('object', 'http://www.wikidata.org/entity/Q37122'), ('label', 'Athena')]
[('object', 'http://www.wikidata.org/entity/Q38012'), ('label', 'Hera')]
[('object', 'http://www.wikidata.org/entity/Q39503'), ('label', 'Artemis')]
[('object', 'http://www.wikidata.org/entity/Q44384'), ('label', 'Hephaestus')]
[('object', 'http://www.wikidata.org/entity/Q41484'), ('label', 'Hermes')]
[('object', 'http://www.wikidata.org/entity/Q41127'), ('label', 'Poseidon')]
[('object', 'http://www.wikidata.org/entity/Q41419'), ('label', 'Hestia')]
[('object', 'http://www.wikidata.org/entity/Q34201'), ('label', 'Zeus')]
[('object', 'http://www.wikidata.org/entity/Q35500'), ('label', 'Aphrodite')]
[('object', 'http://www.wikidata.org/entity/Q40730'), ('label', 'Demeter')]
[('object', 'http://www.wikidata.org/entity/Q40901'), ('label', 'Ares')]
12


In [50]:
# checking 'instance of' property of node 'Apollo'
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q37340 wdt:P31 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q511056'), ('label', 'solar deity')]
[('object', 'http://www.wikidata.org/entity/Q22989102'), ('label', 'Greek deity')]
2


Final query for this task

In [51]:
# write your final query
queryString = """
SELECT ?deity_iri ?label
WHERE
{
   wd:Q41127 wdt:P361 ?twelve_olympians.
   ?twelve_olympians wdt:P527 ?deity_iri.
   
   ?deity_iri sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('deity_iri', 'http://www.wikidata.org/entity/Q37340'), ('label', 'Apollo')]
[('deity_iri', 'http://www.wikidata.org/entity/Q37122'), ('label', 'Athena')]
[('deity_iri', 'http://www.wikidata.org/entity/Q38012'), ('label', 'Hera')]
[('deity_iri', 'http://www.wikidata.org/entity/Q39503'), ('label', 'Artemis')]
[('deity_iri', 'http://www.wikidata.org/entity/Q44384'), ('label', 'Hephaestus')]
[('deity_iri', 'http://www.wikidata.org/entity/Q41484'), ('label', 'Hermes')]
[('deity_iri', 'http://www.wikidata.org/entity/Q41127'), ('label', 'Poseidon')]
[('deity_iri', 'http://www.wikidata.org/entity/Q41419'), ('label', 'Hestia')]
[('deity_iri', 'http://www.wikidata.org/entity/Q34201'), ('label', 'Zeus')]
[('deity_iri', 'http://www.wikidata.org/entity/Q35500'), ('label', 'Aphrodite')]
[('deity_iri', 'http://www.wikidata.org/entity/Q40730'), ('label', 'Demeter')]
[('deity_iri', 'http://www.wikidata.org/entity/Q40901'), ('label', 'Ares')]
12


## Task 8
Compare ancient Greece with ancient Egypt and find how many deities each one has.

In [52]:
# write your queries
# checking 'part of' property of node 'greek deity'
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q22989102 wdt:P361 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q34726'), ('label', 'Greek mythology')]
[('object', 'http://www.wikidata.org/entity/Q855270'), ('label', 'religion in ancient Greece')]
2


In [53]:
# getting all instances of 'egyption mythology' using regex based on the pattern of node 'greek mythology'
queryString = """
SELECT DISTINCT ?subject ?label
WHERE
{
   ?subject wdt:P361 ?object.
   
   FILTER REGEX(?label_object, "Egyptian mythology")
   
   ?subject sc:name ?label.
   ?object sc:name ?label_object.
   
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('subject', 'http://www.wikidata.org/entity/Q145915'), ('label', 'Nekhbet')]
[('subject', 'http://www.wikidata.org/entity/Q58373'), ('label', 'Amun')]
[('subject', 'http://www.wikidata.org/entity/Q667670'), ('label', 'Buchis')]
[('subject', 'http://www.wikidata.org/entity/Q170569'), ('label', 'Atum')]
[('subject', 'http://www.wikidata.org/entity/Q211611'), ('label', 'Anuket')]
[('subject', 'http://www.wikidata.org/entity/Q146313'), ('label', 'Sobek')]
[('subject', 'http://www.wikidata.org/entity/Q191815'), ('label', 'Neith')]
[('subject', 'http://www.wikidata.org/entity/Q129106'), ('label', 'Bastet')]
[('subject', 'http://www.wikidata.org/entity/Q130227'), ('label', 'Aten')]
[('subject', 'http://www.wikidata.org/entity/Q133343'), ('label', 'Hathor')]
[('subject', 'http://www.wikidata.org/entity/Q116273'), ('label', 'Tefnut')]
[('subject', 'http://www.wikidata.org/entity/Q146321'), ('label', 'Ptah')]
[('subject', 'http://www.wikidata.org/entity/Q1126848'), ('label', 'Huh')]
[('

In [54]:
# getting object/data properties of node 'Nekhbet' found from last query
queryString = """
SELECT DISTINCT ?property ?label
WHERE
{
   wd:Q145915 ?property ?object.
   
   ?property sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1417'), ('label', 'Encyclopædia Britannica Online ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1711'), ('label', 'British Museum person or institution ID')]
[('property', 'http://www.wikidata.org/prop/direct/P18'), ('label', 'image')]
[('property', 'http://www.wikidata.org/prop/direct/P21'), ('label', 'sex or gender')]
[('property', 'http://www.wikidata.org/prop/direct/P31'), ('label', 'instance of')]
[('property', 'http://www.wikidata.org/prop/direct/P361'), ('label', 'part of')]
[('property', 'http://www.wikidata.org/prop/direct/P373'), ('label', 'Commons category')]
[('property', 'http://www.wikidata.org/prop/direct/P5905'), ('label', 'Comic Vine ID')]
[('property', 'http://www.wikidata.org/prop/direct/P5986'), ('label', 'Getty Iconography Authority ID')]
[('property', 'http://www.wikidata.org/prop/direct/P646'), ('label', 'Freebase ID')]
10


In [55]:
# checking 'instance of' property of node 'Nekhbet'
queryString = """
SELECT DISTINCT ?object ?label
WHERE
{
   wd:Q145915 wdt:P31 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q30301495'), ('label', 'legendary bird')]
[('object', 'http://www.wikidata.org/entity/Q146083'), ('label', 'Ancient Egyptian deity')]
2


In [56]:
# getting object/data properties of node 'ancient egyption deity'
queryString = """
SELECT DISTINCT ?property ?label
WHERE
{
   wd:Q146083 ?property ?object.
   
   ?property sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P18'), ('label', 'image')]
[('property', 'http://www.wikidata.org/prop/direct/P2354'), ('label', 'has list')]
[('property', 'http://www.wikidata.org/prop/direct/P244'), ('label', 'Library of Congress authority ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2596'), ('label', 'culture')]
[('property', 'http://www.wikidata.org/prop/direct/P268'), ('label', 'Bibliothèque nationale de France ID')]
[('property', 'http://www.wikidata.org/prop/direct/P279'), ('label', 'subclass of')]
[('property', 'http://www.wikidata.org/prop/direct/P361'), ('label', 'part of')]
[('property', 'http://www.wikidata.org/prop/direct/P373'), ('label', 'Commons category')]
[('property', 'http://www.wikidata.org/prop/direct/P4212'), ('label', 'PACTOLS thesaurus ID')]
[('property', 'http://www.wikidata.org/prop/direct/P508'), ('label', 'BNCF Thesaurus ID')]
[('property', 'http://www.wikidata.org/prop/direct/P646'), ('label', 'Freebase ID')]
[('prop

In [57]:
# checking 'part of' property of node 'ancient egyption deity'
queryString = """
SELECT DISTINCT ?object ?label
WHERE
{
   wd:Q146083 wdt:P361 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q205740'), ('label', 'Egyptian mythology')]
[('object', 'http://www.wikidata.org/entity/Q108092253'), ('label', 'Ancient Egyptian pantheon')]
2


In [58]:
# getting all instances of node 'ancient egyption deity'
# here the actual count is 156
queryString = """
SELECT DISTINCT ?subject ?label
WHERE
{
   ?subject wdt:P31 wd:Q146083.
   
   ?subject sc:name ?label.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('subject', 'http://www.wikidata.org/entity/Q145915'), ('label', 'Nekhbet')]
[('subject', 'http://www.wikidata.org/entity/Q58373'), ('label', 'Amun')]
[('subject', 'http://www.wikidata.org/entity/Q2620491'), ('label', 'Werethekau')]
[('subject', 'http://www.wikidata.org/entity/Q170569'), ('label', 'Atum')]
[('subject', 'http://www.wikidata.org/entity/Q211611'), ('label', 'Anuket')]
[('subject', 'http://www.wikidata.org/entity/Q146313'), ('label', 'Sobek')]
[('subject', 'http://www.wikidata.org/entity/Q1252904'), ('label', 'Ra')]
[('subject', 'http://www.wikidata.org/entity/Q130227'), ('label', 'Aten')]
[('subject', 'http://www.wikidata.org/entity/Q133343'), ('label', 'Hathor')]
[('subject', 'http://www.wikidata.org/entity/Q845614'), ('label', 'Maahes')]
[('subject', 'http://www.wikidata.org/entity/Q116273'), ('label', 'Tefnut')]
[('subject', 'http://www.wikidata.org/entity/Q1936121'), ('label', 'Min-Kamutef')]
[('subject', 'http://www.wikidata.org/entity/Q1627813'), ('label', 

In [59]:
# getting all instances of node 'ancient egyption deity'
# here the actual count is 281 when I remove ?label from last query
queryString = """
SELECT DISTINCT ?subject
WHERE
{
   ?subject wdt:P31 wd:Q146083.
}
LIMIT 50 
"""

print("Results")
x=run_query(queryString)

Results
[('subject', 'http://www.wikidata.org/entity/Q145915')]
[('subject', 'http://www.wikidata.org/entity/Q58373')]
[('subject', 'http://www.wikidata.org/entity/Q2620491')]
[('subject', 'http://www.wikidata.org/entity/Q3337608')]
[('subject', 'http://www.wikidata.org/entity/Q3358339')]
[('subject', 'http://www.wikidata.org/entity/Q170569')]
[('subject', 'http://www.wikidata.org/entity/Q211611')]
[('subject', 'http://www.wikidata.org/entity/Q146313')]
[('subject', 'http://www.wikidata.org/entity/Q1252904')]
[('subject', 'http://www.wikidata.org/entity/Q130227')]
[('subject', 'http://www.wikidata.org/entity/Q133343')]
[('subject', 'http://www.wikidata.org/entity/Q845614')]
[('subject', 'http://www.wikidata.org/entity/Q116273')]
[('subject', 'http://www.wikidata.org/entity/Q1936121')]
[('subject', 'http://www.wikidata.org/entity/Q2828997')]
[('subject', 'http://www.wikidata.org/entity/Q3273638')]
[('subject', 'http://www.wikidata.org/entity/Q1627813')]
[('subject', 'http://www.wikidata

In [60]:
# getting all instances of node 'greek deity'
# here the actual count is 241
queryString = """
SELECT DISTINCT ?subject ?label
WHERE
{
   ?subject wdt:P31 wd:Q22989102.
   
   ?subject sc:name ?label.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('subject', 'http://www.wikidata.org/entity/Q5081127'), ('label', 'Acmon')]
[('subject', 'http://www.wikidata.org/entity/Q366533'), ('label', 'Adranus')]
[('subject', 'http://www.wikidata.org/entity/Q3362052'), ('label', 'Pandora')]
[('subject', 'http://www.wikidata.org/entity/Q6612'), ('label', 'Charon')]
[('subject', 'http://www.wikidata.org/entity/Q2144233'), ('label', 'Pheme')]
[('subject', 'http://www.wikidata.org/entity/Q1120723'), ('label', 'Pheme')]
[('subject', 'http://www.wikidata.org/entity/Q1862189'), ('label', 'Paean')]
[('subject', 'http://www.wikidata.org/entity/Q1775444'), ('label', 'Phosphorus')]
[('subject', 'http://www.wikidata.org/entity/Q178417'), ('label', 'Eos')]
[('subject', 'http://www.wikidata.org/entity/Q59697'), ('label', 'Hesperus')]
[('subject', 'http://www.wikidata.org/entity/Q1084868'), ('label', 'Lyssa')]
[('subject', 'http://www.wikidata.org/entity/Q1294659'), ('label', 'Telesphorus')]
[('subject', 'http://www.wikidata.org/entity/Q18423351'), 

In [61]:
# getting all instances of node 'greek deity'
# here the actual count is 252 when I remove ?label from last query
queryString = """
SELECT DISTINCT ?subject
WHERE
{
   ?subject wdt:P31 wd:Q22989102.
}
"""

print("Results")
x=run_query(queryString)

Results
[('subject', 'http://www.wikidata.org/entity/Q5081127')]
[('subject', 'http://www.wikidata.org/entity/Q366533')]
[('subject', 'http://www.wikidata.org/entity/Q3362052')]
[('subject', 'http://www.wikidata.org/entity/Q6612')]
[('subject', 'http://www.wikidata.org/entity/Q2144233')]
[('subject', 'http://www.wikidata.org/entity/Q1120723')]
[('subject', 'http://www.wikidata.org/entity/Q1862189')]
[('subject', 'http://www.wikidata.org/entity/Q1775444')]
[('subject', 'http://www.wikidata.org/entity/Q178417')]
[('subject', 'http://www.wikidata.org/entity/Q59697')]
[('subject', 'http://www.wikidata.org/entity/Q1084868')]
[('subject', 'http://www.wikidata.org/entity/Q1294659')]
[('subject', 'http://www.wikidata.org/entity/Q18423351')]
[('subject', 'http://www.wikidata.org/entity/Q185670')]
[('subject', 'http://www.wikidata.org/entity/Q37340')]
[('subject', 'http://www.wikidata.org/entity/Q83223')]
[('subject', 'http://www.wikidata.org/entity/Q850227')]
[('subject', 'http://www.wikidata.o

Final query for this task

In [62]:
# write your final query
queryString = """
SELECT DISTINCT ?deity ?label (COUNT(*) AS ?num_deity)
WHERE
{
   values ?deity { wd:Q146083 wd:Q22989102 }
   ?subject wdt:P31 ?deity.
   
   ?deity sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('deity', 'http://www.wikidata.org/entity/Q146083'), ('label', 'Ancient Egyptian deity'), ('num_deity', '281')]
[('deity', 'http://www.wikidata.org/entity/Q22989102'), ('label', 'Greek deity'), ('num_deity', '252')]
2
